In [ ]:
from collections import defaultdict
import time
import random
import dynet as dy
import numpy as np

In [22]:
w2i = defaultdict(lambda: len(w2i))
t2i = defaultdict(lambda: len(t2i))
UNK = w2i["<unk>"]

In [15]:
# Functions to read in the corpus
w2i = defaultdict(lambda: len(w2i))
t2i = defaultdict(lambda: len(t2i))
UNK = w2i["<unk>"]
def read_dataset(filename):
  with open(filename, "r") as f:
    for line in f:
      tag, words = line.lower().strip().split(" ||| ")
      yield ([w2i[x] for x in words.split(" ")], t2i[tag])

In [16]:
# Read in the data
train = list(read_dataset("../data/classes/train.txt"))
w2i = defaultdict(lambda: UNK, w2i)
dev = list(read_dataset("../data/classes/test.txt"))
nwords = len(w2i)
ntags = len(t2i)

In [17]:
train[0][1]

0

In [18]:
# Start DyNet and define trainer
model = dy.Model()
trainer = dy.AdamTrainer(model)

In [19]:
# Define the model
W_sm = model.add_lookup_parameters((nwords, ntags)) # Word weights
b_sm = model.add_parameters((ntags))                # Softmax bias

In [20]:
# A function to calculate scores for one value
def calc_scores(words):
  dy.renew_cg()
  score = dy.esum([dy.lookup(W_sm, x) for x in words])
  b_sm_exp = dy.parameter(b_sm)
  return score + b_sm_exp

In [21]:
for ITER in range(100):
  # Perform training
  random.shuffle(train)
  train_loss = 0.0
  start = time.time()
  for words, tag in train:
    my_loss = dy.pickneglogsoftmax(calc_scores(words), tag)
    train_loss += my_loss.value()
    my_loss.backward()
    trainer.update()
  print("iter %r: train loss/sent=%.4f, time=%.2fs" % (ITER, train_loss/len(train), time.time()-start))
  # Perform testing
  test_correct = 0.0
  for words, tag in dev:
    scores = calc_scores(words).npvalue()
    predict = np.argmax(scores)
    if predict == tag:
      test_correct += 1
  print("iter %r: test acc=%.4f" % (ITER, test_correct/len(dev)))

iter 0: train loss/sent=1.5161, time=0.26s
iter 0: test acc=0.3602
iter 1: train loss/sent=1.3404, time=0.23s
iter 1: test acc=0.3787
iter 2: train loss/sent=1.2364, time=0.25s
iter 2: test acc=0.3824
iter 3: train loss/sent=1.1579, time=0.25s
iter 3: test acc=0.4036
iter 4: train loss/sent=1.0935, time=0.24s
iter 4: test acc=0.4059
iter 5: train loss/sent=1.0383, time=0.25s
iter 5: test acc=0.4027
iter 6: train loss/sent=0.9904, time=0.25s
iter 6: test acc=0.4072
iter 7: train loss/sent=0.9498, time=0.29s
iter 7: test acc=0.4131
iter 8: train loss/sent=0.9132, time=0.24s
iter 8: test acc=0.4118
iter 9: train loss/sent=0.8799, time=0.27s
iter 9: test acc=0.4050
iter 10: train loss/sent=0.8500, time=0.23s
iter 10: test acc=0.4154
iter 11: train loss/sent=0.8228, time=0.24s
iter 11: test acc=0.4204
iter 12: train loss/sent=0.7981, time=0.26s
iter 12: test acc=0.4158
iter 13: train loss/sent=0.7754, time=0.25s
iter 13: test acc=0.4186
iter 14: train loss/sent=0.7538, time=0.28s
iter 14: t